In [19]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from docx import Document
from docx.shared import Inches, RGBColor
from matplotlib.dates import DateFormatter


# --- Paths ---
base_dir = "../outputs/portfolio1"
data_dir = os.path.join(base_dir, "data")
analysis_dir = os.path.join(base_dir, "analysis")

# Ensure directories exist
os.makedirs(data_dir, exist_ok=True)
os.makedirs(analysis_dir, exist_ok=True)

# --- Filenames ---
report_name = "portfolio_risk_report.docx"
text_report_name = "portfolio_llm_report.txt"

report_path = os.path.join(analysis_dir, report_name)
text_report_path = os.path.join(analysis_dir, text_report_name)

# --- Load data ---
returns_df = pd.read_csv(os.path.join(data_dir, "returns.csv"), index_col=0)
perf_df = pd.read_csv(os.path.join(data_dir, "performance.csv"), index_col=0)
risk_df = pd.read_csv(os.path.join(data_dir, "risk.csv"), index_col=0)
prices_df = pd.read_csv(os.path.join(data_dir, "prices.csv"))


# Ensure all indices with dates are datetime formatted for plots/tables
returns_df.index = pd.to_datetime(returns_df.index)

# --- Create document ---
doc = Document()
title = doc.add_heading('Portfolio Risk & Performance Report', level=1)
title_run = title.runs[0]
title_run.font.color.rgb = RGBColor(0, 51, 102)
title_run.font.size = Inches(0.4)

doc.add_paragraph(
    f"This report summarizes the portfolio’s performance, risk metrics, and "
    f"underlying holdings based on data from {output_dir}."
)


doc.add_heading('1. Portfolio Summary', level=2)
start_date = pd.to_datetime(returns_df.index.min()).strftime("%m/%d/%Y")
end_date = pd.to_datetime(returns_df.index.max()).strftime("%m/%d/%Y")
doc.add_paragraph(f"Number of Holdings: {len(perf_df)}")
doc.add_paragraph(f"Date Range: {start_date} → {end_date}")


plots_dir = os.path.join(output_dir, "plots")
os.makedirs(plots_dir, exist_ok=True)

# 1. Cumulative portfolio returns
if "PORTFOLIO" in returns_df.columns:
    plt.figure(figsize=(6, 4))
    ax = (1 + returns_df["PORTFOLIO"]).cumprod().plot(color="navy", linewidth=2)
    ax.set_title("Cumulative Portfolio Returns")
    ax.set_xlabel("Date")
    ax.set_ylabel("Cumulative Return")
    ax.xaxis.set_major_formatter(DateFormatter("%m/%d/%Y"))
    plt.xticks(rotation=45, ha='right')
    plt.grid(True)
    cumret_path = os.path.join(plots_dir, "cumulative_returns.png")
    plt.tight_layout()
    plt.savefig(cumret_path, dpi=300)
    plt.close()
    doc.add_picture(cumret_path, width=Inches(6))

# 2. Risk-return scatter
if "Volatility" in perf_df.columns and "Annualized Return" in perf_df.columns:
    plt.figure(figsize=(6, 4))
    plt.scatter(perf_df["Volatility"], perf_df["Annualized Return"], color="steelblue")
    for sym in perf_df.index:
        plt.text(perf_df.loc[sym, "Volatility"], perf_df.loc[sym, "Annualized Return"], sym)
    plt.xlabel("Volatility")
    plt.ylabel("Annualized Return")
    plt.title("Risk-Return Profile")
    plt.grid(True)
    rr_path = os.path.join(plots_dir, "risk_return_scatter.png")
    plt.tight_layout()
    plt.savefig(rr_path, dpi=300)
    plt.close()
    doc.add_picture(rr_path, width=Inches(6))


# ==============================
# Performance Metrics Table
# ==============================
doc.add_page_break()
doc.add_heading('2. Performance Metrics', level=2)
table = doc.add_table(rows=1, cols=len(perf_df.columns) + 1)
table.style = 'Table Grid'

hdr_cells = table.rows[0].cells
hdr_cells[0].text = "Symbol"
for i, col in enumerate(perf_df.columns):
    hdr_cells[i + 1].text = col

for symbol, row in perf_df.iterrows():
    row_cells = table.add_row().cells
    row_cells[0].text = symbol
    for i, val in enumerate(row):
        if isinstance(val, (float, int)):
            row_cells[i + 1].text = f"{val * 100:.2f}%"
        else:
            row_cells[i + 1].text = str(val)


doc.add_page_break()
doc.add_heading('3. Risk Metrics', level=2)

groups = {
    "Volatility": [col for col in risk_df.columns if "Vol" in col],
    "VaR": [col for col in risk_df.columns if "VaR" in col and "CVaR" not in col],
    "CVaR": [col for col in risk_df.columns if "CVaR" in col],
    "Beta": [col for col in risk_df.columns if "Beta" in col],
    "Correlation": [col for col in risk_df.columns if "Corr" in col]
}

for section, cols in groups.items():
    if not cols:
        continue
    doc.add_paragraph(section, style='List Bullet')
    table = doc.add_table(rows=1, cols=len(cols) + 1)
    table.style = 'Table Grid'

    hdr = table.rows[0].cells
    hdr[0].text = "Symbol"
    for i, col in enumerate(cols):
        hdr[i + 1].text = col

    for symbol, row in risk_df.iterrows():
        row_cells = table.add_row().cells
        row_cells[0].text = symbol
        for i, col in enumerate(cols):
            val = row[col]
            if isinstance(val, (float, int)):
                # Scale for percentage-style risk metrics
                if "Beta" in col:
                    row_cells[i + 1].text = f"{val:.2f}"
                else:
                    row_cells[i + 1].text = f"{val * 100:.2f}%"
            else:
                row_cells[i + 1].text = str(val)


doc.add_page_break()
doc.add_heading('4. Analysis Notes', level=2)
doc.add_paragraph(
    "This section includes commentary or automatically generated insights "
    "based on portfolio performance and risk exposure. "
    "All visuals and tables are generated from saved data in outputs/portfolio1."
)


doc.save(report_path)


/var/folders/_m/tt93sf8n4917ybg86fltwhcm0000gn/T/ipykernel_35028/3975742923.py:34: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  returns_df.index = pd.to_datetime(returns_df.index)


In [ ]:
# ==============================
# Generate Text Report for LLMs (as one multiline string)
# ==============================

text_report = f"""
============================================================
PORTFOLIO RISK & PERFORMANCE REPORT (LLM-FRIENDLY VERSION)
============================================================

Data Source Directory: {output_dir}
Evaluation Date Range: {start_date} → {end_date}
Number of Holdings: {len(perf_df)}
============================================================

SECTION 1: PERFORMANCE METRICS
These metrics measure returns, volatility, and efficiency of each stock.
============================================================
Metric Definitions:
 - Total Return: (Final portfolio value / Initial value) - 1
 - Annualized Return: Daily compounded return adjusted for 252 trading days
 - Volatility: Standard deviation of daily returns × sqrt(252)
 - Sharpe Ratio: Annualized Return ÷ Volatility (risk-adjusted return)
 - Max Drawdown: Largest peak-to-trough decline in cumulative returns
 - Upside Capture: Average portfolio return on market up days ÷ market up return
 - Downside Capture: Average portfolio return on market down days ÷ market down return
============================================================
"""

# Add performance metrics
for symbol, row in perf_df.iterrows():
    text_report += f"""
----- {symbol} PERFORMANCE SUMMARY -----
"""
    for metric, val in row.items():
        if isinstance(val, (float, int)):
            if 'Return' in metric or 'Drawdown' in metric or 'Capture' in metric:
                val_str = f"{val * 100:.2f}%"
            else:
                val_str = f"{val:.4f}"
        else:
            val_str = str(val)
        text_report += f"{metric}: {val_str}\n"
    text_report += "============================================================\n"

# Add risk metrics section
text_report += """
SECTION 2: RISK METRICS
These metrics quantify exposure, loss probability, and correlation with the market.
============================================================
Metric Definitions:
 - Volatility: Annualized standard deviation of returns
 - VaR (Value at Risk): Maximum expected daily loss with a given confidence level (e.g. 95%)
 - CVaR (Conditional VaR): Average loss on the worst 5% of days
 - Beta: Covariance(stock, S&P500) ÷ Variance(S&P500) — sensitivity to market moves
 - Correlation with S&P 500: Statistical correlation between stock and market daily returns
============================================================
"""

for symbol, row in risk_df.iterrows():
    text_report += f"""
----- {symbol} RISK SUMMARY -----
"""
    for metric, val in row.items():
        if isinstance(val, (float, int)):
            if "Beta" in metric:
                val_str = f"{val:.2f}"
            else:
                val_str = f"{val * 100:.2f}%"
        else:
            val_str = str(val)
        text_report += f"{metric}: {val_str}\n"
    text_report += "============================================================\n"

# Add portfolio context section (editable block)
text_report += f"""
SECTION 3: PORTFOLIO CONTEXT AND NOTES
============================================================
Performance metrics were computed using daily price data and standard finance formulas.

Risk metrics were computed relative to the S&P 500 index over the same period.

All percentages represent daily or annualized values unless stated otherwise.

Report Generated On: {pd.Timestamp.now().strftime('%m/%d/%Y %H:%M:%S')}

============================================================
"""

# Save to file
text_report_path = os.path.join(output_dir, "portfolio_llm_report.txt")
with open(text_report_path, "w") as f:
    f.write(text_report)

print(f"Text report for LLMs saved to {text_report_path}")


Text report for LLMs saved to ../outputs/portfolio1/analysis/portfolio_llm_report.txt
